In [76]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def extract_protein_name(file_path):
    """
    Extracts the protein name from the file name.
    Assumes the protein name is in the first two underscore-separated tokens,
    where the second token may have additional info (e.g. "-RT") which is removed.
    """
    filename = os.path.basename(file_path)
    tokens = filename.split('_')
    if len(tokens) < 2:
        raise ValueError(f"Filename {filename} does not have enough tokens to extract protein name.")
    token1 = tokens[0]
    token2 = tokens[1].split('-')[0]
    return f"{token1}_{token2}"

def synchronize_videos(videos_dict, time_multiplier, grid_shape, channel, fps_output=30.0, output_file_path="combined_video.avi"):
    """
    Synchronizes multiple video files based on a common time axis.
    
    Instead of specifying an absolute time_step, the function computes the base time step 
    (i.e. the minimum effective frame duration among the videos) and then multiplies it 
    by the provided time_multiplier.
    
    IMPORTANT: To ensure the movies are never cut short, the function uses the maximum total
    effective time among the videos. That way the output covers the entire duration of all movies;
    for any video that ends before the others, its last frame is repeated.
    
    Parameters:
        videos_dict (dict): Keys are video file paths, values are effective frame durations (in seconds)
                            for each video.
        time_multiplier (float): Multiplier to the base time step (min of frame durations). 
                                 For maximum resolution, use 1.
        grid_shape (tuple): (rows, columns) specifying the grid layout for the output video.
        fps_output (float, optional): Output video playback frames per second. Defaults to 30.0.
        output_file_path (str, optional): A file path whose directory is used for saving the output video.
                                          The final file name will include the protein names.
        
    Returns:
        None. The output video is saved to a file in the specified directory.
        Also prints the total playback duration (in seconds) of the output grid video.
    """
    # Convert videos_dict keys and values to lists
    file_paths = list(videos_dict.keys())
    frame_durations = list(videos_dict.values())
    
    n_videos = len(file_paths)
    grid_cells = grid_shape[0] * grid_shape[1]
    if grid_cells < n_videos:
        raise ValueError("Grid shape is too small for the number of videos provided.")
    
    # Extract protein names for output naming.
    protein_names = [extract_protein_name(fp) for fp in file_paths]
    proteins_combined = "-".join(protein_names)
    
    # Build the final output file name using the output file path's directory.
    out_dir = os.path.dirname(output_file_path)
    final_output_file = os.path.join(out_dir, f"{proteins_combined}_{channel}_synced.avi")
    
    # Open all videos to get info.
    caps = [cv2.VideoCapture(fp) for fp in file_paths]
    
    # Calculate each video's total time.
    # --- NEVER CUT SHORT: Use the maximum total time so that the output covers the full duration of all movies.
    total_times = []
    num_frames_list = []  # store total frames for each video
    for cap, fd in zip(caps, frame_durations):
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        num_frames_list.append(num_frames)
        total_times.append(num_frames * fd)
    total_time_sync = max(total_times)
    
    # Compute base time step as the minimum effective frame duration among videos.
    base_time_step = min(frame_durations)
    # Final time step is base multiplied by the given multiplier.
    time_step = base_time_step * time_multiplier
    
    # Determine the number of output frames.
    num_output_frames = int(total_time_sync / time_step)
    
    # Calculate playback duration of the output video.
    playback_duration_sec = num_output_frames / fps_output
    print(f"Movie will be {playback_duration_sec:.2f} seconds long.")
    
    # Read the first frame from each video to get dimensions.
    dimensions = []
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, frame = cap.read()
        if not ret:
            raise ValueError("Failed to read the first frame from one or more videos.")
        h, w = frame.shape[:2]
        dimensions.append((h, w))
    
    # Define a common cell size using the maximum height and width among all videos.
    cell_height = max(h for h, w in dimensions)
    cell_width  = max(w for h, w in dimensions)
    
    # Calculate output video dimensions based on grid shape.
    output_frame_height = grid_shape[0] * cell_height
    output_frame_width  = grid_shape[1] * cell_width
    
    # Define the codec and create the VideoWriter object for the output video.
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out = cv2.VideoWriter(final_output_file, fourcc, fps_output, (output_frame_width, output_frame_height))
    
    # Process frames: for each time step, compute the corresponding frame from each video.
    for i in tqdm(range(num_output_frames), desc="Processing frames", unit="frame"):
        current_time = i * time_step
        frames = []
        for idx, (cap, fd) in enumerate(zip(caps, frame_durations)):
            frame_idx = int(current_time / fd)
            if frame_idx >= num_frames_list[idx]:
                frame_idx = num_frames_list[idx] - 1
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap.read()
            if not ret:
                frame = np.zeros((dimensions[idx][0], dimensions[idx][1], 3), dtype=np.uint8)
            frame = cv2.resize(frame, (cell_width, cell_height))
            frames.append(frame)
        
        while len(frames) < grid_cells:
            black = np.zeros((cell_height, cell_width, 3), dtype=np.uint8)
            frames.append(black)
        
        grid_rows = []
        for r in range(grid_shape[0]):
            row_frames = frames[r * grid_shape[1] : (r + 1) * grid_shape[1]]
            row_combined = np.hstack(row_frames)
            grid_rows.append(row_combined)
        combined_frame = np.vstack(grid_rows)
        
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()
    
    print(f"Output saved as: {final_output_file}")

# Example usage:
output_file_path = "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/"

videos = {
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/output_data_/movies/K401_160nM-RT_Rep1_cy5_120fps_2616frames.avi": 60,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/output_data_/movies/K401_80nM-RT_Rep1_cy5_120fps_2616frames.avi": 60,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/output_data_/movies/K401_40nM-RT_Rep1_cy5_120fps_2616frames.avi": 60,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/output_data_/movies/K401_20nM-RT_Rep1_cy5_120fps_2616frames.avi": 60,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/output_data_/movies/K401_10nM-RT_Rep1_cy5_120fps_2616frames.avi": 60,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/output_data_/movies/K401_5nM-RT_Rep1_cy5_120fps_2616frames.avi": 60,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/output_data_/movies/K401_2p5nM-RT_Rep1_cy5_120fps_2616frames.avi": 60,
    # "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/output_data_/movies/K401_1p25nM-RT_Rep1_cy5_120fps_2616frames.avi": 60,
}


# videos = {
#     "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/101324-k401-titration-rt/2p5TMB-1ulDNA_/output_data_/movies/K401_1p25nM-RT_Rep1_cy5_120fps_2616frames.avi": 60,
#     "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/111624-C-E-G-RT/2p5ulTMB-0p5MT-1ulDNA_/output_data_/movies/C_1p25nM_Rep1_cy5_60fps_743frames.avi": 150,
#     "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/111624-C-E-G-RT/2p5ulTMB-0p5MT-1ulDNA_/output_data_/movies/G_1p25nM_Rep1_cy5_60fps_743frames.avi": 150,

# }


videos = {
    "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/100624-kif3-titration-RT/2p5ulTMB-1ulDNAXnM_/output_data/movies/combined_heatmap_movie_cy5_120fps_1800frames.avi": 8,
    "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/110324-D_titration-RT/2p5TMB-1ulDNA_1/output_data_/movies/combined_heatmap_movie_cy5_24fps_338frames.avi": 64,
}

# For maximum temporal resolution, use a multiplier of 1.
# Increase the multiplier to reduce the number of output frames.
time_multiplier = 8
grid_shape = (2, 1)

synchronize_videos(videos, time_multiplier, grid_shape, channel="cy5", fps_output=30, output_file_path=output_file_path)

Movie will be 11.27 seconds long.


Processing frames:   0%|          | 0/338 [00:00<?, ?frame/s]

Processing frames: 100%|██████████| 338/338 [01:00<00:00,  5.59frame/s]

Output saved as: ../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/combined_heatmap-combined_heatmap_cy5_synced.avi


Movie will be 11.27 seconds long.


Processing frames: 100%|██████████| 338/338 [01:47<00:00,  3.15frame/s]


Output saved as: ../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/Kif3_80nM-D_80nM_cy5_synced.avi
Movie will be 11.27 seconds long.


Processing frames: 100%|██████████| 338/338 [01:46<00:00,  3.18frame/s]


Output saved as: ../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/Kif3_40nM-D_40nM_cy5_synced.avi
Movie will be 11.27 seconds long.


...

Output saved as: ../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/Kif3_5nM-D_5nM_cy5_synced.avi
Movie will be 11.27 seconds long.


Processing frames: 100%|██████████| 338/338 [01:54<00:00,  2.96frame/s]


Output saved as: ../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/Kif3_2p5nM-D_2p5nM_cy5_synced.avi
Movie will be 11.27 seconds long.


Processing frames: 100%|██████████| 338/338 [01:55<00:00,  2.92frame/s]

Output saved as: ../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/Kif3_1p25nM-D_1p25nM_cy5_synced.avi


In [73]:
concentrations = ["160"]

for conc in concentrations:
    videos = {
        f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/100624-kif3-titration-RT/2p5ulTMB-1ulDNAXnM_/output_data_/movies/Kif3_{conc}nM_1-RT_Rep1_GFP_120fps_1800frames.avi": 8,
        f"../../../../Thomson Lab Dropbox/David Larios/activedrops/main/110324-D_titration-RT/2p5TMB-1ulDNA_1/output_data_/movies/D_{conc}nM_Rep1_GFP_24fps_338frames.avi": 64,
    }

    # For maximum temporal resolution, use a multiplier of 1.
    # Increase the multiplier to reduce the number of output frames.
    time_multiplier = 8
    grid_shape = (1, 2)

    synchronize_videos(videos, time_multiplier, grid_shape, channel="GFP", fps_output=30, output_file_path=output_file_path)

Movie will be 11.27 seconds long.


Processing frames: 100%|██████████| 338/338 [01:22<00:00,  4.12frame/s]

Output saved as: ../../../../Thomson Lab Dropbox/David Larios/activedrops/main/all/movies/Kif3_160nM-D_160nM_GFP_synced.avi


In [ ]:
dna_concs = []